In [54]:
import requests
import json
import csv
import time
import os.path

In [57]:
'''Convert ms time from API to local time'''
import datetime

# Given timestamp in milliseconds
timestamps = [1744449900, 1744559100]

for ts in timestamps:
    # Convert to seconds
    
    # Convert to a readable datetime (local time)
    readable_time = datetime.datetime.fromtimestamp(ts)
    print(f"Local Time: {ts} -> {readable_time}")

Local Time: 1744449900 -> 2025-04-12 05:25:00
Local Time: 1744559100 -> 2025-04-13 11:45:00


In [79]:
'''Define needed variables'''
# API key used to access summary jsons
runeliteAPI = "https://prices.runescape.wiki/api/v1/osrs/"

# Example exestion for time series query: /timeseries?timestep=5m&id=4151
timeStep = "5m"
itemID = 4151

runeliteAPIQuery = runeliteAPI + f"timeseries?timestep={timeStep}&id={itemID}"

print(f"Query being used: {runeliteAPIQuery}")

# User agent tag used to let wiki know why I am using their API. 
userAgent = {
    'User-Agent': 'Machine Learning Practice',
    'From': 'sliverwall (discord ID)'  # discord ID
}

# Define where data will be written to
fileName = f"runelite_data_{timeStep}_{itemID}"
outputDir = "data/runelite/"
fileExe = ".csv"
filePath = outputDir + fileName + fileExe

# Define headers for output file
colNames = ["itemID", "timeRecorded", "timestamp","avgHighPrice", "avgLowPrice", "highPriceVolume", "lowPriceVolume"]

Query being used: https://prices.runescape.wiki/api/v1/osrs/timeseries?timestep=5m&id=4151


In [82]:
'''getQuery function takes a url request and header dict, then returns a formatted json file'''
def getQuery(url, headers):
    # Get time data was loaded in
    current_timestamp = int(time.time())
    # Get get request for json 
    r = requests.get(url, headers=headers)
    # use json loader to format output
    json_data = json.loads(r.text)
    # return the data and when that data was queried
    return json_data, current_timestamp

'''formatQuery takes a json_data set then formats it according to the runeliteAPI timeseries data then returns the results'''
def formatQuery(json_data, timestamp):
    itemID = json_data['itemId']
    samples = []
    # Loop through data key from queried data
    for item in json_data['data']:
        entry = (
            itemID,
            timestamp,
            item['timestamp'],
            item['avgHighPrice'],
            item['avgLowPrice'],
            item['highPriceVolume'],
            item['lowPriceVolume']
        )
        samples.append(entry)
    # Return
    return samples
def runeliteToCSV(samples, filePath, colNames, appendIfPossible=True,): 
    # Check if we need to append or write
    # Append if path exists and append is on
    if os.path.isfile(filePath) and appendIfPossible:
        writeMode = 'a'
        msg = "appended"
    else:
        writeMode = 'w'
        msg = "written"
        
    with open(filePath, mode=writeMode, newline='') as f:
        # Create csv writer
        csvWriter = csv.writer(f, delimiter = ',')
        # Write column names if not appending
        if writeMode == 'w':
            csvWriter.writerow(colNames)
            
        # Write out item data
        for entry in samples:
            csvWriter.writerow(entry)
    print(f"Output {msg} to {filePath}")
                        

In [ ]:
'''Output section. Runs main logic'''
# Collect data
data, current_timestamp = getQuery(url=runeliteAPIQuery,
                headers=userAgent)

# Format data
samples = formatQuery(json_data=data, timestamp=current_timestamp)

# Write data to output file
runeliteToCSV(samples=samples, filePath=filePath, colNames=colNames) 